In [2]:
import pandas as pd
import re

# CSVファイルを読み込む
df = pd.read_csv('sample.csv')

# "科目名"列に"S評価"が含まれる行を特定
s_evaluation_subjects = df[df['科目名'].str.contains('S評価', na=False)]

# 削除する科目名をログとして表示
if not s_evaluation_subjects.empty:
    print("削除する科目名:")
    print(s_evaluation_subjects['科目名'])

# "S評価"を含む行を削除
df = df[~df['科目名'].str.contains('S評価', na=False)]

# "時限"列を表示
print(df['時限'])

# 学期曜日時限の列を分割する関数
def split_term_day_period(value):
    # 学期を抽出
    term_patterns = r'(春学期|春クォーター|夏クォーター|秋学期|冬クォーター|冬クォーター|通年)'
    terms = re.findall(term_patterns, value)
    
    # 例外処理: 学期がない、または複数ある場合
    if len(terms) != 1:
        return None, None, None  # 例外として処理
    
    term = terms[0]
    
    # 曜日を抽出
    days = ''.join(re.findall(r'(月|火|水|木|金|土|日)', value))
    days = days if days else '無'
    
    # 時限を抽出し、全角を半角に変換し、範囲を埋める
    periods = re.findall(r'(\d+)', value)
    if periods:
        periods = list(map(int, periods))  # 全角を半角に変換してintに
        periods = list(range(min(periods), max(periods) + 1))  # 範囲を埋める
    else:
        periods = []
    
    return term, days, periods

# 新しい列を追加
df['学期'], df['曜日'], df['時限'] = zip(*df['学期曜日時限'].map(split_term_day_period))

# 例外データを別のCSVに保存
exception_df = df[df['学期'].isnull()]
exception_df.to_csv('exceptions.csv', index=False)

# 例外を除いたデータを保存
df = df.dropna(subset=['学期'])
df.to_csv('sample_modified.csv', index=False)